<a href="https://colab.research.google.com/github/Anita1017/Abstractive-Text-Summarization/blob/main/T5%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch

     |████████████████████████████████| 1.5MB 5.8MB/s 
     |████████████████████████████████| 2.9MB 44.4MB/s 
     |████████████████████████████████| 890kB 46.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=fd25d1408ac58ca0640cfa5d44878a2213c0470bf6bfe709d68f1098cb716e95
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
#!pip install sentencepiece
% pip install sentencepiece


     |████████████████████████████████| 1.1MB 5.3MB/s 


In [ ]:
###Output for a Sample Input
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

text ="""
and for those looking to step back in time during a weekend getaway , there are plenty of options available at your fingertips . whether you prefer the earthy aesthetic of the atomic age or the bold hues of the brit-pop '90s , airbnb rentals offers places that cater to interior decor of all tastes . the cottage , which sleeps four , features dark wood beamed ceilings and plenty of mid-century furnishings , situated ideally near the town citadel ."""

preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)

Some weights of the model checkpoint at t5-small were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


original text preprocessed: 
 and for those looking to step back in time during a weekend getaway , there are plenty of options available at your fingertips . whether you prefer the earthy aesthetic of the atomic age or the bold hues of the brit-pop '90s , airbnb rentals offers places that cater to interior decor of all tastes . the cottage , which sleeps four , features dark wood beamed ceilings and plenty of mid-century furnishings , situated ideally near the town citadel .


Summarized text: 
 and for those looking to step back in time during a weekend getaway, there are plenty of options available at your fingertips. airbnb rentals offers places that cater to interior decor of all tastes.


Import Bert Extractive Output File

In [ ]:

import pandas as pd 
df =pd.read_csv("/content/bert_sum.csv")

In [ ]:
len(df)

11489

In [ ]:
prediction=[]

In [ ]:
for t in range(len(df)):
  text=df['text'][t]
  preprocess_text = text.strip().replace("\n","")
  t5_prepared_Text = "summarize: "+preprocess_text

  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
  summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
 
  prediction.append(output)

In [ ]:
import numpy as np

In [ ]:
target=[]
for i in range(len(prediction)):
  target.append(df.summary[i])

In [ ]:
Prediction =np.array(prediction)
Target=np.array(target)


Framing a new DataFrame for Model Evalution

In [ ]:
df=pd.DataFrame({'PREDICTION_SUM':Prediction,'TARGET_SUM':Target})

In [ ]:
from google.colab import files

df.to_csv('T5_sum.csv')
files.download('T5_sum.csv')